## Intro to Faster Computation
### or some insights into speeding up your Python code
Pavel SORIANO


## What is this talk about?

* Get an overview on how to make code run faster with a focus on Python.
    * Making use of our language-of-choice inherent features
    * Passing to lower-level speedups
    * Using parallel computation

### Why?
* We have the technology!
* Large datasets
* Costly algorithms 

## Talk Outline

1. Overview of a machine learning task as use case
2. Some guidelines to (better) programming Python 
3. Some solutions to add C/C++ code inside our Python code
4. Parallel programming in Python

 ## Data Science Reminder
* __Build software__ able to predict and describe
* Using __Machine Learning__ (ML) techniques  

* Typical approaches:
    * __Supervised Learning__: Regression, Classification
    * __Unsupervised Learning__: Clustering

## Parametric vs. Non-parametric
* Parametric
    * Often faster to use
    * Make assumptions about the nature of the data distributions
* Non-parametric
    * More flexible,
    * Often computationally intractable for large datasets

## k - Nearest Neighbors (kNN)
* Simple non-parametric classifier
* Instance-based (no need to generalize)

* Given a labelled dataset $D$,  a new data point $x$ and an integer value $k$,  the main three steps of kNN are:
    1. Measure the similarity (or distance) between $x$ and each point in $D$.
    2. Sort the points according to the calculated similarity
    3. Assign $x$ the majority class found in the top $k$ points


## Task Definition
* We define a syntetic dataset with 750 samples, 20 features and 2 classes.
* The objective is to classify these samples using a supervised machine learning algorithm

In [1]:
%load_ext line_profiler
%load_ext Cython
from sklearn import datasets
import numpy as np

X, Y = datasets.make_classification(n_samples=750, n_features=20,
                                    n_informative=2, n_redundant=2, n_classes=2)

In [3]:
print(X.shape)
print
print(X[:5,])

(750, 20)
[[ 1.75761714e-01 -7.34399585e-01  4.23749769e-01 -1.57847620e+00
  -2.40050190e+00 -1.05980771e-02  8.93842289e-01  2.21004088e+00
   8.58749273e-01 -2.57273607e-01 -1.59658890e+00 -1.07569127e+00
  -8.17482141e-01  2.51642658e-01  6.29761703e-01 -5.32027925e-01
  -1.13744286e+00 -5.97860959e-01 -1.26669464e-01 -1.26635441e+00]
 [-2.39234675e+00 -1.41695243e+00  3.51350023e-01  2.71789856e+00
  -3.21624738e-02 -1.28219452e+00  6.16381907e-01 -1.00570238e+00
   1.31146338e+00  9.92597089e-01 -1.22039655e-01 -1.64125494e+00
  -5.94194711e-01 -2.19571095e+00 -3.56528357e-01  1.49355762e+00
  -3.34339435e-03  1.07060287e+00 -1.60216243e+00  1.32087383e+00]
 [-2.41161698e+00  1.30430122e+00  8.92254726e-01 -2.23926390e-01
   6.44256672e-01 -1.25329229e+00  5.83741510e-01  4.68005940e-01
   2.80521500e-01  1.61384001e+00 -8.61480666e-01  2.84676172e+00
   3.63659505e-01 -2.24721034e+00 -1.04759837e+00  9.76176006e-02
   5.55006873e-01 -1.39963660e+00 -1.52358774e+00  1.46600520e+0

In [5]:
print(Y.shape)
print
print(Y)

(750,)
[0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1
 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0
 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1
 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1
 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1
 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1
 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1
 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 1 1 0
 1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 1
 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1
 0 1 1 1 1 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1
 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1
 1 0 0 0 0 0 0 1 0

# First step
Given a labelled dataset $D$,  a new data point $p$ and an integer value $k$,  the main three steps of kNN are:
1. __Measure the Euclidean distance between $p$ and each point in $q \in D$__

    $$d(p,q) = \sqrt{\sum_{i=1}^{n}(p_i-q_i)^2}$$


In [11]:
def euclidean_distance(p,q):
    euclidean_dist = 0
    for i in range(len(p)):
        euclidean_dist += (p[i] - q[i])**2
    return np.sqrt(euclidean_dist)

print(euclidean_distance(X[0,], X[1,]))
print(np.linalg.norm(X[0,] - X[1,]))
    
    

8.454959754738583
8.454959754738583


## Second step

1. Measure the Euclidean distance between $x$ and each point in $D$
2. __Sort the points according to the calculated similarity__


In [16]:
def sort_array(array):
    # Return the sorted array and its indices as a list of tuples : [(index1, value1), (index2,value2)]
    indexed_array = dict(zip(range(len(array)), array))
    return sorted(indexed_array.items(), key=lambda x:x[1])
    
# test 
print(sort_array([4,3,2,1,0]))



[(4, 0), (3, 1), (2, 2), (1, 3), (0, 4)]


## Third step

1. Measure the similarity (or distance) between $x$ and each point in $D$.
2. Sort the points according to the calculated similarity
3. __Assign $x$ the majority class found in the top $k$ points__

In [17]:
def assign_class(k, ordered_distances, Y):
    from collections import Counter

    closest_ids = [s[0] for s in ordered_distances[:k]]
    closest_labels = Y[closest_ids]
    chosen_label = Counter(closest_labels).most_common()[0][0]
    return chosen_label
#test
# assign_class(3, [(3, 1), (2, 2), (1, 3), (0, 4)], np.array([1,2,2,2,3]))

## Putting it all together


In [22]:
def kNN(X, Y, k, test_points):
    test_classes = []
    for p in test_points:
        distances = []
        for i in range(X.shape[0]):
            if not np.array_equal(p, X[i, ]):
                distances.append(euclidean_distance(X[i, ], p))
        ordered_distances = sort_array(distances)
        chosen_label = assign_class(k, ordered_distances, Y)
        test_classes.append(chosen_label)
    return test_classes

# print kNN(X, Y, 3, X[101:120,])
    
    

In [23]:
# Test its accuracy. Just because why not
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.6)
Y_hat = kNN(X_train, Y_train, 3, X_test)

print(accuracy_score(Y_test, Y_hat))


0.8377777777777777


## Measure to improve: Profiling
* Very important first step before improving our code
* Find these hotspots in the code
    * What is slow? 
    * What is using too much memory? 
    * Maybe too much network or disk I/O?
* We want to do the least amount of work to get the biggest practical performance gain. 

## Profiling
### timeit module
* Coarse profiling
* Runs our code in a loop, multiple times.
* Returns the best averaged result (in term of time) of all the loop repetitions
* We want the best result because the slower results may be due to the computer busy with other things
* By default it runs 10 loops (swith _n_) with 3 repetitions (switch _r_)


Always profile before optimizing. You could spend time optimizing a function that is in fact not taking that much time as others

In [24]:
%timeit -n 3 -r 3 kNN(X_train, Y_train, 5, X_test)
# In pure Python
# python -m timeit -n 3 -r 3 "kNN(X_train, Y_train, 3, X_test)"

3.39 s ± 109 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Profiling
### cProfile module
* Built-in profiling tool
* Hooks into CPython virtual machine
* We can get a high-level overview of the time spent in our code
* Function-based profiling
* Add some overhead making code run slower



In [25]:
%prun -s cumulative -D program.prof kNN(X_train, Y_train, 5, X_test)
# In pure Python
# python -m cProfile -s cumulative kNN.py

 
*** Profile stats marshalled to file 'program.prof'. 


![title](img/cProfile_out)


* The entry point of our kNN function is indicated in line 2. It takes a total of 2.7 seconds. As expected, this function is only called once, shown in ncalls.

* Inside kNN, euclidean_distance takes 1.598 seconds to compute and it is called 135,000 times, which makes sense because we have a 750 points dataset, we split it into 450 training (40%) and 300 testing (60%). So 450x300 = 135k

* In second place of slowness, we have the array_equal call, taking 0.85 seconds. Actually the next four lines (all, _all, reduce, asarray) are part of this call. 

* Way below we find our sort_array function, which seems to be not very slow and in last place we have assign_class, with 0.20 cumtime

* So, with this knowledge we can focus in the first two slow parts: euclidean_distance and the array_equal


* cumtime is the time spent in the function/method including the time spent in the functions/methods that it calls
* tottime is the time spent in the function/method excluding the time spent in the functions/methods that it calls.

## Profiling
### SnakeViz
Browser based grpahical viewer of cProfile's output

In [26]:

#pip install snakeviz
!snakeviz program.prof 

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fhome%2Fpavel%2Fcode%2FPythonPerformanceTalk%2Fprogram.prof
^C

Bye!


## Profiling
### line_profiler module
* Profiles individual functions on a line-by-line basis. 
* After analyzing the code with cProfile, we can go deeper with line_profiler
* Very important tool for Python CPU-based profling


In [27]:
%lprun -f kNN -f euclidean_distance -f sort_array -f assign_class kNN(X_train, Y_train, 5, X_test)
# In Python (needs decorators):
# kernprof.py -l -v my_script.py

![title](img/line_prof0.png)

![title](img/line_prof1.png)

dynamic lookup at every loop. Compiling and type specialization would improve this.

![title](img/line_profiler2.png)

## Optimizing Cycle

1. Create a real use case benchmark. A big enough chunk of data that does not take forever to compute
2. Run a profiler (high-level: cProfile, in-detail: line_profiler)
3. Look for hotspots. If one or two functions take more than (roughly) 60% of the time, there is hope.
4. Apply optimization
5. Re-run benchmark (luckily you will see improvements)
6. Optional but should not be optional: Test that your algorithm is still correct after modifications


## Some faster Python guidelines

* While using matrices, or vectors, do not deal with items line-by-line or point-by-point (try to avoid `for` loops)
* If possible, use sets, and by extension, dictionaries to get fast access to data
* Still, if your data has already a defined order, stay with lists and/or tuples
* Pre-allocate your `numpy` arrays
* If your data is static, it is better to use tuples
* Take advantage of generators. They use less memory and may avoid costly operations while dealing only with punctual values of interest.
* Try to avoid conditions inside _for_ loops
* Take advantage of elementary built-in functions (`sorted`, `min`, `max`) and look for higher-level well-known solutions to your problems (`numpy`, `scikit-learn`)


In [28]:
def i_hate_broadcasting():
    a = np.array([1.0, 2.0, 3.0])
    b = np.array([2.0, 2.0, 2.0])
    a * b

def i_love_broadcasting():
    a = np.array([1.0, 2.0, 3.0])
    b = 2.0
    a * b



## Improving the code somehow

* Remove the _if_ condition from the `for` loop. Check for distance greater than zero in a list comprehension
* Eliminate one `for` loop by using Numpy broadcasting


In [15]:
# Recall the original time duration
%timeit kNN(X_train, Y_train, 5, X_test)

1 loops, best of 3: 2.1 s per loop


In [29]:
def assign_class2(k, ordered_distances, Y):
    """
    We check in the list comprehension that the distance is greater than zero
    """
    from collections import Counter

    closest_ids = [s[0] for s in ordered_distances[:k] if s[1] > 0]
    closest_labels = Y[closest_ids]
    chosen_label = Counter(closest_labels).most_common()[0][0]
    return chosen_label

def kNN2(X, Y, k, test_points):
    """
    We remove the if condition at each iteration!
    """
    test_classes = []
    for p in test_points:
        distances = []
        for i in range(X.shape[0]):
            distances.append(euclidean_distance(X[i, ], p))
        ordered_distances = sort_array(distances)
        chosen_label = assign_class2(k, ordered_distances, Y)
        test_classes.append(chosen_label)
    return test_classes

In [17]:
%timeit kNN2(X_train, Y_train, 5, X_test)

1 loops, best of 3: 1.47 s per loop


In [18]:
def kNN3(X, Y, k, test_points):
    """
    We eliminate one for loop by using broadcasting and calculating the distance between each dataset
    point and the test point.
    """
    test_classes = []
    for p in test_points:
        distances = np.sqrt(((X[:, np.newaxis] - p) ** 2).sum(axis=-1))
        ordered_distances = sort_array(distances)
        chosen_label = assign_class2(k, ordered_distances, Y)
        test_classes.append(chosen_label)
    return test_classes


In [19]:
%timeit kNN3(X_train, Y_train, 5, X_test)

1 loops, best of 3: 502 ms per loop


## Stop Here!
* From a code-maintenance point of view, it is wise to stop here
* We should appreaciate the quick wins and consider the diminishing returns with the extra effort involved
* In 3 months (or 3 days), our smart code snippets will be hard to interpret even for ourselves



## Enter the Machine Code
* Python is dinamically typed, so at each moment the virtual machine has a difficult time optimizing the machine code as it does not know what is the type of data used in the program
* A recurrent approach in most of the most important machine learning libraries.
* Python is used as a "glue" (front-end) while the computations are done in C or Fortran
* Once we are using solid algorithms, data structures, efficient code, we may go on with compiling code
* We compile the slowest parts of our code in machine code (using C/LLVM) within Python


## Enter the Machine Code

* Works well when you have a lot of loops (because you can't avoid them)
* Reach close-to-C performance
* Not very useful for I/O operations, string manipulation and calls to external modules
* Adds a considerable layer of complexity to your code
* Very active area. Changes/new developments appear constantly

## The Tools

|   Name  | Compiler | Numpy compatible? |
|:-------:|:--------:|:-----------------:|
|  Cython |    AOT   |        yes        |
|  Numba  |    JIT   |        some        |
| Pythran |    AOT   |        yes        |
|   PyPy  |    JIT   |         no        |

* Ahead Of Time (AOT) compiler
    * Creates a static library which is called every time you use the compiled code
    * Scikit-learn, scipy, and others are installed with these static libraries 
* Just In Time (JIT) compiler
    * Most of the times all it requires is a decorator and your code is compiled on the fly at execution
    * It may slowdown considerably during the startup of your program as it is compiling the code to be used during execution. Later references to compiled code should run faster.
* Trade-off between speed vs simplicity. Although JIT solutions seem to be faster or at least as fast as AOT compilers.

## Cython
* Cython is a programming language for writing C extensions in Python
* Adds type annotations to Python allowing fast compiled code
* Cython is a superset of Python, a kind of hybrid between Python and C
* Simple to start using it but gets complex as more requirements are needed
* Supports OpenMP allowing (as the name indicates) multi-processes to run in parallel




In [20]:
%%cython
cimport cython
cimport numpy as np
from libc.math cimport sqrt

def assign_class4(k, ordered_distances, Y):
    """
    We check in the list comprehension that the distance is greater than zero
    """
    from collections import Counter

    closest_ids = [s[0] for s in ordered_distances[:k] if s[1] > 0]
    closest_labels = Y[closest_ids]
    chosen_label = Counter(closest_labels).most_common()[0][0]
    return chosen_label

def sort_array4(array):
    indexed_array = dict(zip(range(len(array)), array))
    return sorted(indexed_array.items(), key=lambda x:x[1])

@cython.boundscheck(False)
@cython.wraparound(False)
def euclidean_distance4(np.ndarray[double, ndim=1] p, np.ndarray[double, ndim=1] q):
    cdef double euclidean_dist = 0
    cdef unsigned int i
    for i in range(p.shape[0]):
        euclidean_dist += (p[i] - q[i])**2
    return sqrt(euclidean_dist)


@cython.boundscheck(False)
@cython.wraparound(False)

def kNN4(X, Y, k, test_points):
    test_classes = []
    for p in test_points:
        distances = []
        for i in range(X.shape[0]):
            distances.append(euclidean_distance4(X[i, ], p))
        ordered_distances = sort_array4(distances)
        chosen_label = assign_class4(k, ordered_distances, Y)
        test_classes.append(chosen_label)
    return test_classes

/usr/share/anaconda/lib/python2.7/site-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [21]:
%timeit kNN4(X_train, Y_train, 5, X_test)

10 loops, best of 3: 194 ms per loop


## Numba
* JIT compiler that transforms Python into optimized machine code using the LLVM Intermediate Representation 
* Does not use g++ or gcc as Cython.
* It only needs some decorators to work
* Also allows parallel execution

In [22]:
from numba import jit

@jit
def euclidean_distance5(p,q):
    euclidean_dist = 0
    for i in range(len(p)):
        euclidean_dist += (p[i] - q[i])**2
    return np.sqrt(euclidean_dist)

def kNN5(X, Y, k, test_points):
    """
    We use numba!
    """
    test_classes = []
    for p in test_points:
        distances = []
        for i in range(X.shape[0]):
            distances.append(euclidean_distance5(X[i, ], p))
        ordered_distances = sort_array(distances)
        chosen_label = assign_class2(k, ordered_distances, Y)
        test_classes.append(chosen_label)
    return test_classes

In [23]:
%timeit kNN5(X_train, Y_train, 5, X_test)

1 loops, best of 3: 165 ms per loop


## Pythran
* Subset of the Python language, with a focus on scientific computing
* Specifically oriented towards scientific computing
* Allows parallel execution
* No need to "dumb" down your code. You can use Numpy oriented functions directly
* Made in France

## Not enough? Larger dataset? Let's parallelize
* We have multi-core computers everywhere. We should be making them work for us (sometimes)
* Best case scenario: $n$ times speedup, where $n$ is your number of cores. Still, some overhead is created and it surely will slowdown
* Python has OS-native threads (real deal threads). Still, they are bound bu the Global Interpreter Lock (GIL)
* This means that only one thread may interact with Python objects at a time
* The GIL is avoided usually by:
    * Using processes: we run Python interpreters in parallel, each running in a private memory space with its own GIL and running code in series
    * By releasing the GIL with low-level solutions (Cython, Numba, etc)


## Not enough? Larger dataset? Let's parallelize
### Some considerations
* Libraries used should be thread-safe to avoid hard-to-debug problems
* Amdahl's Law: Be aware that most of the execution time is taken by serial operations. If only a small part of the code can be parallelized, then it is of little importance how many processors you use. It won't run a lot faster.
* Hyperthreading = ~30% of a real extra core
* Much easier to work on \*nix-based systems

## Type of parallel problems
* Fine coarsed: tasks must communicate many times per second
* Grain coarsed: they do not communicate many times per second
* Embarassingly parallel: no communication (or very rarely) between tasks



## Multiprocessing 
* Python module that allows us to spawn processes
* Effectively side-steps the Global Interpreter Lock by using subprocesses instead of threads
* It pickles parameters and functions and pass them around. Try not to pass a lot of data... 
* Keep the functions as simple as possible

## Joblib
* Lightweight pipelining in Python
* Provides a helper class to write parallel for loops using multiprocessing
* Transparent and fast disk-caching of output values
* The vision is to provide tools to easily achieve better performance and reproducibility


## Multiprocessing Example (with a much larger dataset)


In [38]:

X, Y = datasets.make_classification(n_samples=5000, n_features=20,
                                    n_informative=2, n_redundant=2, n_classes=2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.6)

X_g = X_train
Y_g = Y_train
k_g = 5


def single_distance(p):
    distances = np.sqrt(((X_g[:, np.newaxis] - p) ** 2).sum(axis=-1))
    ordered_distances = sort_array(distances)
    chosen_label = assign_class2(k_g, ordered_distances, Y_g)
    return chosen_label


def kNN6(test_points):
        
    import multiprocessing
    """
    We try the parallel version!
    """
    pool = multiprocessing.Pool(processes = 4)
    test_classes = pool.map(single_distance, test_points)
    return test_classes


In [39]:
%timeit kNN6(X_test)

1 loops, best of 3: 10 s per loop


## Now let's try the fastest single core version with Numba (same large dataset)



In [36]:
%timeit kNN5(X_train, Y_train, 5, X_test)

KeyboardInterrupt: 

## Parallelizing by releasing the GIL
* If we use Cython it is as easy as calling the appropriate commands with _nogil_ and _prange_
![title](img/nogil.png)
* While using Numba 
![title](img/prange_numba.png)

## Me, parallelizing ... in Java ¯\\_(ツ)_/¯
Show my use case here. Not data science per se but still useful

## Other promising tool: Dask
>Dask enables parallel computing through task scheduling and blocked algorithms. This allows developers to write complex parallel algorithms and execute them in parallel either on a modern multi-core machine or on a distributed cluster.
![title](img/dask.png)

## Requirements

* Anaconda 
* Ipython?
* PyCharm?
* Unix system (easier parallelization with processes)
 

## Conclusion

* Premature optimization is the root of all evil. Still, no excuses for lazy, slow code
* Still, do not waste time optimizing something that is not optimizable. Better to look for other algorithms/logic
* Unit testing should be part of our experiments. We may introduce errors to our functions in our quest for speed
* Keeping it readable is the top priority, in 2 weeks you won't remember what your clever self from the past did
* Don't get addicted to it


## References

* High Performance Python. Micha Gorelick and Ian Ozvald
* Python for Data Analysis. Wes McKinney
* Course from V. Miele http://pbil.univ-lyon1.fr/members/miele/tutoriel/
* Travis Oliphant blog http://technicaldiscovery.blogspot.fr/2011/06/speeding-up-python-numpy-cython-and.html
* Jake VanderPlas blog https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/
* Scipy performance tips http://scipy.github.io/old-wiki/pages/PerformanceTips
* Using Python for performance computing http://scipy.github.io/old-wiki/pages/PerformancePython
* Scikit-learn performance tips http://scikit-learn.org/stable/developers/performance.html
* Machine Learning:  A Probabilistic Perspective. Kevin P. Murphy
* A Course in Machine Learning. Hal Daume III


# Let us get to work with XGBoost-Dask